# Generalized Neural Network
* A denoising autoencoder that learns the user's ratings and implicit ratings

In [1]:
name = "GNN";
residual_alphas = ["UserItemBiases"];

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
BLAS.set_num_threads(Threads.nthreads());

In [5]:
device = gpu;

## Data preparation

In [6]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas)
# column accesses are faster than row accesses, so we make this an (item, user) matrix 
const R = sparse(
    training.item,
    training.user,
    convert.(Float32, training.rating),
    maximum(training.item) + 1,  # leave room for unseen users and items
    maximum(training.user) + 1,
);
const n_items, n_users = size(R);

In [7]:
counts = zeros(Float32, n_users, Threads.nthreads())
@tprogress Threads.@threads for u in training.user
    counts[u, Threads.threadid()] += 1
end
counts = sum(counts, dims = 2);

Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.36 μs/it)


In [8]:
function get_data(R, split, j, train)
    # inputs are the user's ratings (unseen shows get mapped to zero) + implicit ratings + heterogenous features
    # during training, outputs are the user's ratings + implicit ratings 
    # during inference, outputs are the user's rating + implicit rating for a held out item on their list

    # handle users and items that aren't in the training set
    u = min(split.user[j], n_users)
    i = min(split.item[j], n_items)

    # ratings
    X1 = collect(R[:, u])
    X1[i] = 0
    # implicit ratings
    X2 = copy(X1)
    X2[X2.!=0] .= 1
    # heterogeneous features
    count = convert(Float32, max(counts[u] - 1, 0) / n_items)
    X3 = [count, sqrt(count), count^2]
    X = vcat(X1, X2, X3)

    # outputs
    Y1 = zeros(eltype(X1), length(X1))
    Y2 = zeros(eltype(X2), length(X2))
    if train
        mask = X2 .!= 0
        Y1[mask] .= X1[mask]
        Y2[mask] .= X2[mask]
    else
        Y1[i] = split.rating[j]
        Y2[i] = 1
    end

    (X, Y1, Y2)
end

function get_batch(R, split, block_size, train)
    items = rand(1:length(split.rating), block_size)
    data = [[] for j = 1:Threads.nthreads()]
    Threads.@threads for i = 1:length(items)
        push!(data[Threads.threadid()], get_data(R, split, items[i], train))
    end
    X = Flux.batch([data[t][i][1] for t = 1:Threads.nthreads() for i = 1:length(data[t])])
    Y1 = Flux.batch([data[t][i][2] for t = 1:Threads.nthreads() for i = 1:length(data[t])])
    Y2 = Flux.batch([data[t][i][3] for t = 1:Threads.nthreads() for i = 1:length(data[t])])
    [(X, (Y1, Y2))] |> device
end;

In [9]:
# custom split layer
struct Split{T}
    paths::T
end
Split(paths...) = Split(paths)
Flux.@functor Split
(m::Split)(x::AbstractArray) = map(f -> f(x), m.paths)

In [10]:
# inputs are the user's ratings for all shows (unseen shows get mapped to zero) + implicit ratings + heterogenous features
# outputs are the user's ratings for all shows (unseen shows get mapped to zero), implicit ratings
# we will train ratings using mse on observed shows, and implicit ratings via crossentropy loss
n_items = size(R)[1]
encoder = Chain(
    Dense(n_items + n_items + 3, 512, relu),
    Dense(512, 256, relu),
    Dense(256, 128, relu),
)
rating_decoder = Chain(Dense(128, 256, relu), Dense(256, 512, relu), Dense(512, n_items))
implicit_decoder = Chain(Dense(128, 256, relu), Dense(256, 512, relu), Dense(512, n_items))
m = Chain(Dropout(0.5), encoder, Split(rating_decoder, implicit_decoder)) |> device
ps = Flux.params(m);
m

Chain(
  Dropout(0.5),
  Chain(
    Dense(33965, 512, relu),            # 17_390_592 parameters
    Dense(512, 256, relu),              # 131_328 parameters
    Dense(256, 128, relu),              # 32_896 parameters
  ),
  Split(
    Tuple(
      Chain(
        Dense(128, 256, relu),          # 33_024 parameters
        Dense(256, 512, relu),          # 131_584 parameters
        Dense(512, 16981),              # 8_711_253 parameters
      ),
      Chain(
        Dense(128, 256, relu),          # 33_024 parameters
        Dense(256, 512, relu),          # 131_584 parameters
        Dense(512, 16981),              # 8_711_253 parameters
      ),
    ),
  ),
)                   # Total: 18 arrays, 35_306_538 parameters, 134.685 MiB.

## Training

In [11]:
function rating_loss(ŷ, y)
    # only compute loss on items the user has seen
    mask = y .!= 0
    Flux.mse(ŷ[mask], y[mask])
end

implicit_loss(ŷ, y) = Flux.logitcrossentropy(ŷ, y)

function implicit_loss(ŷ, y, mask)
    # we're predicting a held out series
    # so we zero out items in the user's training set
    ŷ[mask] .= -1e3
    implicit_loss(ŷ, y)
end

function loss_components(x, y, train)
    ŷ = m(x)
    if train
        return (rating_loss(ŷ[1], y[1]), implicit_loss(ŷ[2], y[2]))
    else
        mask = (x.!=0)[1:n_items, :]
        return (rating_loss(ŷ[1], y[1]), implicit_loss(ŷ[2], y[2], mask))
    end
end;

In [12]:
opt = ADAM();

In [13]:
# best_loss = Inf
# patience = 10
# iters_without_improvement = 0
# continue_training = true
# iters = 0

# function evalcb(R, split, train)
#     losses = []
#     @showprogress for epoch = 1:100
#         push!(losses, loss_components(get_batch(R, split, 128, train)[1]..., train))
#     end
#     reduce(.+, losses) ./ length(losses)
# end

# const training_baseline_loss = evalcb(R, training, true)
# const inference_baseline_loss = evalcb(R, training, false);
# function loss(x, y)
#     sum(loss_components(x, y, true) ./ training_baseline_loss)
# end

# function evalcb()
#     # print losses and perform early stopping
#     testmode!(m)
#     @debug "iteration: $iters"
#     training_losses = evalcb(R, training, true)
#     training_loss = sum(training_losses ./ training_baseline_loss)
#     @debug "training losses: $(training_losses) -> $(training_loss)"
#     inference_losses = evalcb(R, validation, false)
#     loss = sum(inference_losses ./ inference_baseline_loss)
#     @debug "validation losses: $(inference_losses) -> $(loss)"
#     if loss < best_loss
#         global best_loss = loss
#         global iters_without_improvement = 0
#         BSON.@save "../../data/alphas/$name/model.bson" m
#     else
#         global iters_without_improvement += 1
#         if iters_without_improvement >= patience
#             global continue_training = false
#         end
#     end
#     trainmode!(m)
# end

# throttled_cb = Flux.throttle(evalcb, 600);

In [14]:
# while continue_training
#     batch = get_batch(R, training, 128, true)
#     Flux.train!(loss, ps, batch, opt, cb = throttled_cb)
#     iters += 1
# end

## Write predictions

In [15]:
function get_data(R, u)
    # ratings
    X1 = collect(R[:, u])
    # implicit ratings
    X2 = copy(X1)
    X2[X2.!=0] .= 1
    # heterogeneous features
    count = convert(Float32, max(counts[u] - 1, 0) / n_items)
    X3 = [count, sqrt(count), count^2]
    vcat(X1, X2, X3)
end

function get_batch(R, users)
    data = [[] for j = 1:Threads.nthreads()]
    Threads.@threads for i = 1:length(users)
        push!(data[Threads.threadid()], get_data(R, users[i]))
    end
    X = Flux.batch([data[t][i] for t = 1:Threads.nthreads() for i = 1:length(data[t])])
    X |> device
end;

In [34]:
function gmodel(users, items)
    ratings = zeros(length(users))
    implicit = zeros(length(users))
    deduped_users = collect(Set(users))
    batch(arr, n) = [arr[i:min(i + n - 1, end)] for i = 1:n:length(arr)]
    batches = batch(deduped_users, 128)
    @tprogress Threads.@threads for i = 1:length(batches)
        b = batches[i]
        alpha = m(get_batch(R, b)) |> cpu
        user_to_idx = Dict(zip(b, 1:length(b)))
        for j = 1:length(users)
            if users[j] in keys(user_to_idx)
                idx = user_to_idx[users[j]]
                ratings[j] = alpha[1][items[j], idx]
                implicit[j] = alpha[2][items[j], idx]
            end
        end
    end
    ratings, implicit
end;

In [17]:
BSON.@load "../../data/alphas/$name/model.bson" m
testmode!(m);
BLAS.set_num_threads(1) # gmodel internally multithreads

In [37]:
full_df = reduce(cat, [training, validation, get_residuals("test", residual_alphas)]);
ratings, implicit = gmodel(full_df.user, full_df.item)
sparse_preds = sparse(full_df.user, full_df.item, ratings);

Progress:   9%|██▍                        |  ETA: 0:08:29 ( 2.52  s/it)

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
     [1] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90;4mIn[36]:15[0m[90m [inlined][39m
     [2] [0m[1m(::var"#979#threadsfor_fun#61"{Vector{Int64}, Vector{Int64}, Vector{Vector{Int64}}, Vector{Float64}, Vector{Float64}, UnitRange{Int64}})[22m[0m[1m([22m[90monethread[39m::[0mBool[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m./[39m[90;4mthreadingconstructs.jl:81[0m
     [3] [0m[1m(::var"#979#threadsfor_fun#61"{Vector{Int64}, Vector{Int64}, Vector{Vector{Int64}}, Vector{Float64}, Vector{Float64}, UnitRange{Int64}})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[35mMain[39m [90m./[39m[90;4mthreadingconstructs.jl:48[0m

In [30]:
function model(users, items)
    preds = zeros(length(users))
    @tprogress Threads.@threads for j = 1:length(preds)
        preds[j] = sparse_preds[users[j], items[j]]
    end
    preds
end;

In [31]:
write_predictions(model, save_training = true)

Progress: 100%|███████████████████████████| Time: 0:00:06 ( 1.57 μs/it)
[ Info: 20220128 19:17:17 training set: RMSE 1.0727818466225378 MAE 0.794094792337937 R2 0.3124673599625506
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 1.36 μs/it)
[ Info: 20220128 19:17:19 validation set: RMSE 1.1790108198041087 MAE 0.8694588002806078 R2 0.18144326004122924
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 1.36 μs/it)
